In [1]:
import tensorflow as tf
import numpy as np
import setGPU
from importlib import reload
import dadrah.playground.test_gradient_tape as tegrta
import dadrah.selection.loss_strategy as ls
import dadrah.selection.discriminator as disc

setGPU: Setting GPU to: 0


In [2]:
tf.__version__

'2.2.0'

In [3]:
x = tf.random.uniform([3000, 1], minval=1, maxval=100, dtype=tf.float32)
y = tf.random.uniform([3000,1], minval=1, maxval=10, dtype=tf.float32)

In [4]:
x.shape

TensorShape([3000, 1])

In [5]:
quantile = 0.1
strategy = ls.combine_loss_min

In [9]:
reload(disc)
discriminator = disc.QRDiscriminator(quantile=quantile, loss_strategy=strategy, epochs=10, n_nodes=30)
discriminator.fit(x, y)
#regressor = tegrta.QuantileRegressionV2() 
#model = regressor.make_model(x_mean_var=(np.mean(x), np.var(x)))
print(discriminator.model.summary())


Start of epoch 0
Training loss (for one batch) at step 0: 22.723190307617188
Training loss (for one batch) at step 2: 18.726512908935547
Training loss (for one batch) at step 4: 17.877185821533203
Training loss (for one batch) at step 6: 14.043351173400879
Training loss (for one batch) at step 8: 13.787345886230469
Training loss (for one batch) at step 10: 9.477657318115234
Training loss (for one batch) at step 12: 8.797708511352539
Training loss (for one batch) at step 14: 7.82716178894043
Training loss (for one batch) at step 16: 8.134836196899414
Training loss (for one batch) at step 18: 8.824692726135254
Training loss (for one batch) at step 20: 8.91861343383789
Training loss (for one batch) at step 22: 7.822970390319824

Start of epoch 1
Training loss (for one batch) at step 0: 7.62371301651001
Training loss (for one batch) at step 2: 7.972781181335449
Training loss (for one batch) at step 4: 7.719485282897949
Training loss (for one batch) at step 6: 7.957101821899414
Training lo

Training loss (for one batch) at step 6: 8.033365249633789
Training loss (for one batch) at step 8: 8.355247497558594
Training loss (for one batch) at step 10: 7.101864814758301
Training loss (for one batch) at step 12: 7.530684471130371
Training loss (for one batch) at step 14: 7.79904317855835
Training loss (for one batch) at step 16: 6.924688816070557
Training loss (for one batch) at step 18: 7.243954658508301
Training loss (for one batch) at step 20: 7.560063362121582
Training loss (for one batch) at step 22: 7.454474449157715

Start of epoch 13
Training loss (for one batch) at step 0: 7.449627876281738
Training loss (for one batch) at step 2: 7.9605536460876465
Training loss (for one batch) at step 4: 7.591211318969727
Training loss (for one batch) at step 6: 8.043828964233398
Training loss (for one batch) at step 8: 8.349967956542969
Training loss (for one batch) at step 10: 7.096415042877197
Training loss (for one batch) at step 12: 7.536911964416504
Training loss (for one batch

Training loss (for one batch) at step 2: 7.967255592346191
Training loss (for one batch) at step 4: 7.609829425811768
Training loss (for one batch) at step 6: 8.04998779296875
Training loss (for one batch) at step 8: 8.330480575561523
Training loss (for one batch) at step 10: 7.11219596862793
Training loss (for one batch) at step 12: 7.528994560241699
Training loss (for one batch) at step 14: 7.797492027282715
Training loss (for one batch) at step 16: 6.901211738586426
Training loss (for one batch) at step 18: 7.234837532043457
Training loss (for one batch) at step 20: 7.567105770111084
Training loss (for one batch) at step 22: 7.443316459655762

Start of epoch 25
Training loss (for one batch) at step 0: 7.448978424072266
Training loss (for one batch) at step 2: 7.959220886230469
Training loss (for one batch) at step 4: 7.595559597015381
Training loss (for one batch) at step 6: 8.05402946472168
Training loss (for one batch) at step 8: 8.336478233337402
Training loss (for one batch) at 


Start of epoch 36
Training loss (for one batch) at step 0: 7.449802398681641
Training loss (for one batch) at step 2: 7.963845252990723
Training loss (for one batch) at step 4: 7.591770172119141
Training loss (for one batch) at step 6: 8.059288024902344
Training loss (for one batch) at step 8: 8.328255653381348
Training loss (for one batch) at step 10: 7.10693359375
Training loss (for one batch) at step 12: 7.533851623535156
Training loss (for one batch) at step 14: 7.781269073486328
Training loss (for one batch) at step 16: 6.886734485626221
Training loss (for one batch) at step 18: 7.230682373046875
Training loss (for one batch) at step 20: 7.568514823913574
Training loss (for one batch) at step 22: 7.438812255859375

Start of epoch 37
Training loss (for one batch) at step 0: 7.448484420776367
Training loss (for one batch) at step 2: 7.958773136138916
Training loss (for one batch) at step 4: 7.600405693054199
Training loss (for one batch) at step 6: 8.052024841308594
Training loss (

Training loss (for one batch) at step 16: 6.903165340423584
Training loss (for one batch) at step 18: 7.242545127868652
Training loss (for one batch) at step 20: 7.576935768127441
Training loss (for one batch) at step 22: 7.434330463409424

Start of epoch 48
Training loss (for one batch) at step 0: 7.448892116546631
Training loss (for one batch) at step 2: 7.962510585784912
Training loss (for one batch) at step 4: 7.60082483291626
Training loss (for one batch) at step 6: 8.049226760864258
Training loss (for one batch) at step 8: 8.335723876953125
Training loss (for one batch) at step 10: 7.110115051269531
Training loss (for one batch) at step 12: 7.5300798416137695
Training loss (for one batch) at step 14: 7.797059535980225
Training loss (for one batch) at step 16: 6.906932830810547
Training loss (for one batch) at step 18: 7.241290092468262
Training loss (for one batch) at step 20: 7.570888996124268
Training loss (for one batch) at step 22: 7.441948890686035

Start of epoch 49
Trainin

Training loss (for one batch) at step 8: 8.331623077392578
Training loss (for one batch) at step 10: 7.108924865722656
Training loss (for one batch) at step 12: 7.531540870666504
Training loss (for one batch) at step 14: 7.793542861938477
Training loss (for one batch) at step 16: 6.902478218078613
Training loss (for one batch) at step 18: 7.236998558044434
Training loss (for one batch) at step 20: 7.571258544921875
Training loss (for one batch) at step 22: 7.4446024894714355

Start of epoch 60
Training loss (for one batch) at step 0: 7.448911666870117
Training loss (for one batch) at step 2: 7.962349891662598
Training loss (for one batch) at step 4: 7.599430084228516
Training loss (for one batch) at step 6: 8.057812690734863
Training loss (for one batch) at step 8: 8.326594352722168
Training loss (for one batch) at step 10: 7.108104228973389
Training loss (for one batch) at step 12: 7.533138751983643
Training loss (for one batch) at step 14: 7.785914421081543
Training loss (for one bat

Training loss (for one batch) at step 2: 7.9642109870910645
Training loss (for one batch) at step 4: 7.60451602935791
Training loss (for one batch) at step 6: 8.050935745239258
Training loss (for one batch) at step 8: 8.33091926574707
Training loss (for one batch) at step 10: 7.110411643981934
Training loss (for one batch) at step 12: 7.530399322509766
Training loss (for one batch) at step 14: 7.798259735107422
Training loss (for one batch) at step 16: 6.910511493682861
Training loss (for one batch) at step 18: 7.236508846282959
Training loss (for one batch) at step 20: 7.564128398895264
Training loss (for one batch) at step 22: 7.450791835784912

Start of epoch 72
Training loss (for one batch) at step 0: 7.449369430541992
Training loss (for one batch) at step 2: 7.963285446166992
Training loss (for one batch) at step 4: 7.5928449630737305
Training loss (for one batch) at step 6: 8.05876350402832
Training loss (for one batch) at step 8: 8.32533073425293
Training loss (for one batch) at


Start of epoch 83
Training loss (for one batch) at step 0: 7.449799537658691
Training loss (for one batch) at step 2: 7.966331481933594
Training loss (for one batch) at step 4: 7.599984645843506
Training loss (for one batch) at step 6: 8.049132347106934
Training loss (for one batch) at step 8: 8.332558631896973
Training loss (for one batch) at step 10: 7.110859394073486
Training loss (for one batch) at step 12: 7.529368877410889
Training loss (for one batch) at step 14: 7.798802375793457
Training loss (for one batch) at step 16: 6.912568092346191
Training loss (for one batch) at step 18: 7.241021156311035
Training loss (for one batch) at step 20: 7.5629682540893555
Training loss (for one batch) at step 22: 7.449703216552734

Start of epoch 84
Training loss (for one batch) at step 0: 7.4501824378967285
Training loss (for one batch) at step 2: 7.969762325286865
Training loss (for one batch) at step 4: 7.612244606018066
Training loss (for one batch) at step 6: 8.042268753051758
Training 

Training loss (for one batch) at step 20: 7.562649726867676
Training loss (for one batch) at step 22: 7.450356960296631

Start of epoch 95
Training loss (for one batch) at step 0: 7.449376106262207
Training loss (for one batch) at step 2: 7.963282108306885
Training loss (for one batch) at step 4: 7.590299606323242
Training loss (for one batch) at step 6: 8.047698974609375
Training loss (for one batch) at step 8: 8.340556144714355
Training loss (for one batch) at step 10: 7.1055073738098145
Training loss (for one batch) at step 12: 7.531401634216309
Training loss (for one batch) at step 14: 7.794197082519531
Training loss (for one batch) at step 16: 6.90949821472168
Training loss (for one batch) at step 18: 7.239975452423096
Training loss (for one batch) at step 20: 7.564706325531006
Training loss (for one batch) at step 22: 7.447644233703613

Start of epoch 96
Training loss (for one batch) at step 0: 7.449519157409668
Training loss (for one batch) at step 2: 7.965912818908691
Training 

In [10]:
x_test = tf.random.uniform([300, 1], minval=1, maxval=100, dtype=tf.float32)

In [11]:
x_test

<tf.Tensor: shape=(300, 1), dtype=float32, numpy=
array([[18.487127 ],
       [16.861074 ],
       [92.28928  ],
       [96.92668  ],
       [50.121624 ],
       [33.68681  ],
       [28.721767 ],
       [40.233143 ],
       [54.551426 ],
       [97.98378  ],
       [97.22862  ],
       [63.698807 ],
       [98.870125 ],
       [20.362291 ],
       [12.642586 ],
       [77.735146 ],
       [21.457336 ],
       [ 5.1104445],
       [89.220665 ],
       [56.86901  ],
       [74.38422  ],
       [80.01396  ],
       [ 2.852221 ],
       [89.45097  ],
       [84.226715 ],
       [91.27842  ],
       [41.463398 ],
       [20.920607 ],
       [33.423325 ],
       [22.43203  ],
       [12.715768 ],
       [26.771852 ],
       [76.642624 ],
       [31.216616 ],
       [42.329857 ],
       [33.90782  ],
       [13.357864 ],
       [36.023785 ],
       [23.12603  ],
       [33.20478  ],
       [44.853573 ],
       [94.57649  ],
       [ 3.6008983],
       [43.379917 ],
       [53.584095 ],
     

In [26]:
y = discriminator.predict(x_test)

In [27]:
y

array([[0.49534723],
       [0.49543893],
       [0.5177956 ],
       [0.5150038 ],
       [0.50528795],
       [0.5119459 ],
       [0.5148262 ],
       [0.50412494],
       [0.49396366],
       [0.49425113],
       [0.49432507],
       [0.5104157 ],
       [0.51002014],
       [0.49757567],
       [0.4959273 ],
       [0.51492584],
       [0.5120544 ],
       [0.5072611 ],
       [0.49691176],
       [0.5027422 ],
       [0.5035956 ],
       [0.49532726],
       [0.49896654],
       [0.51544756],
       [0.49678046],
       [0.51170695],
       [0.50514245],
       [0.49750474],
       [0.49870756],
       [0.50688076],
       [0.49390173],
       [0.4954791 ],
       [0.49583963],
       [0.4950637 ],
       [0.4950526 ],
       [0.49715355],
       [0.4964343 ],
       [0.49631926],
       [0.50843173],
       [0.5025207 ],
       [0.4958775 ],
       [0.49552667],
       [0.51316285],
       [0.49467617],
       [0.5092292 ],
       [0.50852525],
       [0.49821994],
       [0.497

In [28]:
weights = model.get_weights()

In [29]:
model.save('./my_model.h5')

In [30]:
reload(tegrta)
loaded_model = tf.keras.models.load_model('./my_model.h5', custom_objects={'FeatureNormalization': tegrta.FeatureNormalization})

In [31]:
loaded_weights = loaded_model.get_weights()

In [32]:
weights[0]

array([[-0.52151966,  0.34217453,  0.01405535,  0.59535474,  0.69920963,
        -0.1658983 , -0.44183353, -0.02300867, -0.06862972, -0.00430826,
        -0.24678671,  0.7089294 ,  0.0510585 ,  0.01305069, -0.27448598,
        -0.06475051, -0.4706956 , -0.4577948 ,  0.19247283, -0.01148568]],
      dtype=float32)

In [33]:
loaded_weights[0]

array([[-0.52151966,  0.34217453,  0.01405535,  0.59535474,  0.69920963,
        -0.1658983 , -0.44183353, -0.02300867, -0.06862972, -0.00430826,
        -0.24678671,  0.7089294 ,  0.0510585 ,  0.01305069, -0.27448598,
        -0.06475051, -0.4706956 , -0.4577948 ,  0.19247283, -0.01148568]],
      dtype=float32)

In [34]:
for i in range(len(weights)):
    assert np.allclose(weights[0], loaded_weights[0])

In [35]:
y_loaded = loaded_model.predict(x_test)

In [36]:
assert np.allclose(y, y_loaded)